In [ ]:
import geopandas as gpd
import folium

In [ ]:
data = gpd.read_file('filtered dataset/crimes_filtered.shp')

In [ ]:
unique_crimes = data['ofns_desc'].unique()
unique_crimes

In [ ]:
useful_crimes = data[data['ofns_desc'].isin(['HARRASSMENT 2', "SEX CRIMES", "RAPE",
            'MURDER & NON-NEGL. MANSLAUGHTER', 'KIDNAPPING & RELATED OFFENSES',
            'FELONY SEX CRIMES', 'KIDNAPPING'])]

useful_crimes.to_file("filtered dataset/crimes_filtered.shp", driver="ESRI Shapefile")

In [ ]:
manz = gpd.read_file('nyct2020_23b/nyct2020.shp')
manz.to_crs('EPSG:4326', inplace=True)
manz

### 1. Importaciones y preprocesamiento